In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
df=pd.read_csv("superstore.csv")
tempdf=df
print(len(tempdf))

Removing Miscellaneous(Unwanted) columns:

In [ ]:
tempdf.pop("Unnamed: 0")
tempdf.pop("记录数")
print(tempdf.columns)

Rearraging the columns for convenience and creating a Index column (Row.ID)

In [ ]:
columns = [
    # Order Details
    "Row.ID", "Order.ID", "Order.Date", "weeknum", "Year", "Ship.Date", "Order.Priority", "Ship.Mode",    
    # Customer Details
    "Customer.ID", "Customer.Name", "Segment",    
    # Location Details
    "City", "State", "Region", "Country",   
    # Product Details
    "Product.ID", "Product.Name", "Category", "Sub.Category",    
    # Sales and Financials
    "Quantity", "Sales", "Discount", "Profit",    
    # Market Details
    "Market", "Market2"
]

tempdf=tempdf[columns].sort_values(by="Row.ID", ascending=True)
tempdf.isna().any()
tempdf["Row.ID"]=tempdf["Row.ID"].interpolate()
tempdf[tempdf["Row.ID"].isna()]

#setting the dataframe index to show the row number
tempdf.reset_index()
tempdf.set_index(np.arange(0,len(tempdf)), inplace=True)
print(tempdf)


Cleaning the dataset:

In [ ]:
#filling up the Market and Market2 columns
M1=tempdf["Market"].dropna().unique().tolist()
M2=tempdf["Market2"].dropna().unique().tolist()
M2.append("North America")
#print(M1,"\n", M2)

for i in range(len(M1)): 
    tempdf.loc[tempdf["Market"].isna() & (tempdf["Market2"]==M2[i]), "Market"] = M1[i]
    tempdf.loc[tempdf["Market2"].isna() & (tempdf["Market"]==M1[i]), "Market2"] = M2[i]

print(tempdf.iloc[1,0])

cleaning Location related information

In [ ]:
c=pd.read_csv("E:/MTU/WLS_Linux_Projects/Project/superstore_data/cities.csv")
city_state_country=c.loc[:,["name", "state_name", "country_name"]]

In [ ]:
print(len(tempdf[tempdf["Country"].isna()]))
print(len(tempdf[tempdf["State"].isna()]))
print(len(tempdf[tempdf["City"].isna()]))

In [ ]:

# Creating a mapping dictionary 
state_to_country = dict(zip(city_state_country['state_name'], city_state_country['country_name']))
city_to_country = dict(zip(city_state_country['name'], city_state_country['country_name']))
city_to_state = dict(zip(city_state_country['state_name'], city_state_country['name']))

# mapping using np.where
tempdf['Country'] = np.where(tempdf['Country'].isna(), tempdf['State'].map(state_to_country), tempdf['Country'])
tempdf['Country'] = np.where(tempdf['Country'].isna(), tempdf['City'].map(city_to_country), tempdf['Country'])

tempdf['State'] = np.where(tempdf['State'].isna(), tempdf['City'].map(city_to_state), tempdf['State'])

print(len(tempdf[tempdf["Country"].isna()]))
print(len(tempdf[tempdf["State"].isna()]))
print(len(tempdf[tempdf["City"].isna()]))

In [ ]:
freq_table = pd.crosstab(tempdf['Order.Priority'], tempdf['Segment'], margins=True, dropna=True, normalize=True) 
round(freq_table*100, 2)
#(freq_table/len(tempdf.dropna()))*100
